In [ ]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import warnings
warnings.simplefilter('ignore')

In [ ]:
sns.set(palette='deep', style='darkgrid', rc={"figure.figsize": (15, 4)})

# Семинар - Задача распознавания рукописных цифр

In [ ]:
from sklearn.datasets import load_digits

### Загрузим данные

In [ ]:
data = load_digits()
print(data['DESCR'])

In [ ]:
X, y = data.data, data.target

In [ ]:
print('В датасете {} объектов и {} признака'.format(X.shape[0], X.shape[1]))

Посмотрим на баланс классов:

In [ ]:
counts = np.unique(y, return_counts=True)
pd.DataFrame(counts[1], counts[0], columns=['counts'])
sns.barplot(counts[0], counts[1])
plt.show()

### Посмотрим на объекты:

In [ ]:
i = np.random.randint(0, X.shape[0])
print('Class name: {}'.format(y[i]))
print(X[i].reshape(8,8))


plt.imshow(X[i].reshape(8,8), cmap='gray_r')
plt.show()

Разделим выборку на две части: обучающую и тестовую

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.7,
                                                    test_size=0.3, 
                                                    shuffle=True,
                                                   random_state=18)
X_train.shape, X_test.shape, y_train.shape, y_test.shape  

## Метод ближайших соседей

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

Зададим классификатор:

In [ ]:
knn = KNeighborsClassifier()

In [ ]:
knn.fit(X_train, y_train)
knn_predictons = knn.predict(X_test)

In [ ]:
preds = pd.DataFrame(y_test, columns=['True'])
preds['knn_pred'] = knn_predictons

In [ ]:
preds.head()

In [ ]:
# Посмотрим долю правильных ответов:
preds[preds['True'] == preds['knn_pred']].shape[0]/preds.shape[0]

In [ ]:
#Тоже самое средставми sklearn:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, knn_predictons)

Постройте график зависимости доли верных ответов на тесте от количества соседей (от 1 до 100):

In [ ]:
# Ваш код здесь: 

<center>
<h4>
Поговорим подобробнее о Валидации на Лекции </h4></center>

## Кросвалидация

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)

In [ ]:
cvs = cross_val_score(knn, X_train, y_train, scoring='accuracy', cv=5)
print('Средняя доля правильных ответов на кросс валидации: %.3f' % np.mean(cvs))

## Поиск оптимальных параметров по сетке

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
kNN_cv = KNeighborsClassifier(n_neighbors=5, n_jobs=-1, )
params = { 
    'metric':['minkowski', 'manhattan'],
    'n_neighbors': range(3, 50),
         }

gcv = GridSearchCV(kNN_cv, param_grid=params, n_jobs=-1, cv=5, scoring='accuracy')
gcv.fit(X_train, y_train)

In [ ]:
print('Лучший скор %.4f' % gcv.best_score_)
print('при метрике %(metric)s и %(n_neighbors)s соседей' % gcv.best_params_)

#### Что получится на тесте?

In [ ]:
accuracy_score(y_test, gcv.predict(X_test))

In [ ]:
gcv_preds = pd.DataFrame(gcv.predict(X_test), columns=['kNN'])


In [ ]:
gcv_preds['True'] = y_test

Посмотрим на те цифры, которые "путает" наш классификатор

In [ ]:
n = gcv_preds[gcv_preds['True'] != gcv_preds['kNN']].shape[0]
confused = gcv_preds[gcv_preds['True'] != gcv_preds['kNN']].index[np.random.randint(0,n)]
example = X_test[confused].reshape(8,8)
print("# :", confused, 
      "True: ", gcv_preds.iloc[confused]['True'],
      "Predictions: ",gcv_preds.iloc[confused]['kNN'])
plt.imshow(example, cmap='gray_r')
plt.show()

In [ ]:
conf = gcv_preds[gcv_preds['True'] != gcv_preds['kNN']]
conf['val'] = 1
conf_pivot = pd.pivot_table(conf, index=['True'], values='val', columns=['kNN'], aggfunc='count').fillna(0)
conf_pivot